In [1]:
#sets a virtually accurate ROI for the exit area AND door and counts the number of people within the region
import os
import cv2
from ultralytics import YOLO
from tracker import Tracker
import numpy as np

#setup video, cap_out, yolo and deepsort
video_path = os.path.join('.', 'data', 'CCTV_in.mp4')
video_out_path = os.path.join('.', 'CCTV_out.mp4')

cap = cv2.VideoCapture(video_path)
ret, frame = cap.read()

cap_out = cv2.VideoWriter(video_out_path, cv2.VideoWriter_fourcc(*'mp4v'), cap.get(cv2.CAP_PROP_FPS),
                          (1280, 720))

model = YOLO("yolov8n.pt") 
tracker = Tracker()
detection_threshold = 0.3
people_movement = {} #dictionary for tracking people - use this to check if the person is incoming or outgoing the exit
frame_count = 0

In [10]:

#function for exit door
def exitDoorROI(frame):
    roi_vertices = np.array([[710, 31], [810,36], [790,220], [700, 215]], np.int32)
    roi_x = 700
    roi_y = 36
    cv2.polylines(frame, [roi_vertices], True, (0, 0, 255), 1)
    cv2.putText(frame, "Exit Door", (roi_x, roi_y - 10), cv2.FONT_HERSHEY_COMPLEX_SMALL, 1, (0, 0, 255), 1)
    # frame_count += 1
    return roi_vertices

#function for exit area ROI
def exitAreaROI():
    roi_vertices = np.array([[520, 340], [640,220], [800,230], [783, 380]], np.int32)
    return roi_vertices

#function for detecting people (if feet within exit area)
def peopleDetectionYOLO(frame, exitAreaVertices, exitDoorVertices):
    results = model(frame)
    detections = []
    human_area_count = 0
    human_door_count = 0

    for result in results:
        for r in result.boxes.data.tolist():
            x1, y1, x2, y2, score, class_id = r #top left and bottom right coordinates of each object
            x1, x2, y1, y2 = int(x1), int(x2), int(y1), int(y2)
            class_id = int(class_id)
            if class_id==0 and score>=detection_threshold:
                    feet_coordinates = (((x1+x2)/2), y2-5)
                    if cv2.pointPolygonTest(exitAreaVertices, feet_coordinates, False) > 0:
                        human_area_count += 1
                    if cv2.pointPolygonTest(exitDoorVertices, feet_coordinates, False) > 0:
                        human_door_count += 1
                    detections.append([x1, y1, x2, y2, score])

    cv2.putText(frame, f"Workers in exit area: {human_area_count}", (5, 25), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255), 1)
    cv2.putText(frame, f"Workers through exit door: {human_door_count}", (5, 60), cv2.FONT_HERSHEY_TRIPLEX, 1, (255, 255, 255), 1)
    return detections

#function for tracking people
def peopleTrackingDeepSort(frame, detections, exitDoorVertices, exitAreaVertices):
    tracker.update(frame, detections)
    for track in tracker.tracks:
        bbox = track.bbox
        x1, y1, x2, y2 = bbox
        track_id = track.track_id

        if track_id not in people_movement:
            people_movement[track_id] = []
        else:
            people_movement[track_id].append([x1, y1, x2, y2])

        if isPersonEntering(exitDoorVertices, exitAreaVertices, track_id):
            print(f"Person {track_id} is entering the exit area")

        #check why personexiting is not working

        # print(people_movement)

        cv2.rectangle(frame, (int(x1), int(y1)), (int(x2), int(y2)), (255,255,255), 1)
        label = f"Staff Worker {track_id}"
        cv2.putText(frame, label, (int(x1), int(y1) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,255,0), 2)
        
def isPersonEntering(exitDoorVertices, exitAreaVertices, track_id): #Check why this is not working, this works now, check why
    last_coordinates = people_movement.get(track_id, [])[-10:]
    if len(last_coordinates) < 2:
        return False
    feet_coordinates = []
    for coordinate in last_coordinates:
        feet_coordinates.append(((coordinate[0] + coordinate[2]) / 2, coordinate[3] - 5))

    print(track_id, " ", feet_coordinates) #this is working properly
    print(track_id, " current coordinate", feet_coordinates[-1]) #this is working properly
    print(track_id, " previous coordinate", feet_coordinates[-2]) #this is working properly


    if cv2.pointPolygonTest(exitAreaVertices, feet_coordinates[-1], False) > 0:
        if cv2.pointPolygonTest(exitDoorVertices, feet_coordinates[-2], False) > 0:
            return True
    return False

# def isPersonEntering(exitDoorVertices, exitAreaVertices, track_id): #Check why this is not working, this works now, check why
#     # Get the last 10 coordinates of the person based on track
#     last_coordinates = people_movement.get(track_id, [])[-10:]
#     # Check if there are at least 2 coordinates to compare
#     if len(last_coordinates) < 2:
#         return False
#     # Calculate the feet coordinates for the last 10 coordinates
#     feet_coordinates = []
#     for coordinate in last_coordinates:
#         feet_coordinates.append(((coordinate[0] + coordinate[2]) / 2, coordinate[3] - 5))

#     # print(track_id, " ", feet_coordinates) #this is working properly

#     if cv2.pointPolygonTest(exitAreaVertices, feet_coordinates[-1], False) > 0:
#         # print("Person is in exit area")
#         # Check if the person was outside of the exit door in the previous frame
#         if cv2.pointPolygonTest(exitDoorVertices, feet_coordinates[-2], False) > 0:
#             # print("Person is outside exit door")
#             return True
#     return False

# def isPersonExiting(exitDoorVertices, exitAreaVertices, track_id):
#     # Get the last 10 coordinates of the person based on track
#     last_coordinates = people_movement.get(track_id, [])[-10:]
    
#     # Check if there are at least 2 coordinates to compare
#     if len(last_coordinates) < 2:
#         return False

#     # Calculate the feet coordinates for the last 10 coordinates
#     feet_coordinates = []
#     for coordinate in last_coordinates:
#         feet_coordinates.append(((coordinate[0] + coordinate[2]) / 2, coordinate[3] - 5))

#     if cv2.pointPolygonTest(exitDoorVertices, feet_coordinates[-1], False) < 0:
#         # Check if the person was inside the exit area in the previous frame
#         if cv2.pointPolygonTest(exitAreaVertices, feet_coordinates[-2], False) > 0:
#             return False
#     return True

In [3]:
# main loop for processing
while ret:
    frame = cv2.resize(frame, (1280, 720))
    frame_count += 1
    print("Frame Count - ", frame_count)
    exitDoorVertices = exitDoorROI(frame)
    exitAreaVertices = exitAreaROI()
    detections = peopleDetectionYOLO(frame, exitAreaVertices, exitDoorVertices)
    peopleTrackingDeepSort(frame, detections, exitDoorVertices, exitAreaVertices)
    
    cv2.imshow('Video', frame)
    if cv2.waitKey(33) & 0xFF == ord('q'):
        break

    cap_out.write(frame)
    ret, frame = cap.read()

cap.release()
cap_out.release()
cv2.destroyAllWindows()

Ultralytics YOLOv8.0.33  Python-3.7.16 torch-1.13.1+cpu CPU
YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs

0: 384x640 1 person, 1 tv, 71.0ms


Frame Count -  1


Speed: 1.0ms pre-process, 71.0ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 1 tv, 65.8ms
Speed: 1.0ms pre-process, 65.8ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 1 tv, 69.0ms


Frame Count -  2
Frame Count -  3


Speed: 1.0ms pre-process, 69.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 1 tv, 66.5ms
Speed: 1.0ms pre-process, 66.5ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 1 person, 1 tv, 67.2ms


...

1   [(868.4954384744563, 710.3052993276095), (867.8656973560031, 710.0555049733453), (867.0016694000237, 709.9663028412779), (866.3772085603858, 709.9381826889482), (865.5141919153032, 709.9330652224289), (864.5599118863536, 709.9360818900343), (863.9007813761723, 710.6009897757772), (863.3534177297519, 710.8558562961391), (863.1770763805459, 710.9531335334862), (863.4693208386263, 710.3309429563147)]
1  current coordinate (863.4693208386263, 710.3309429563147)
1  previous coordinate (863.1770763805459, 710.9531335334862)
2   [(721.6505169106199, 253.5141351370708), (720.7318641902755, 253.82508219115294), (720.4311010460568, 253.87764976877526), (720.0381874259505, 253.83733574683487), (719.2801668131926, 255.72716023236518), (718.7041047058532, 259.01000457146574), (716.5692065191026, 263.4729889892184), (714.1579493613824, 267.7376490264557), (712.6224117574945, 269.9585764693253), (710.7749116933182, 273.34516661853604)]
2  current coordinate (710.7749116933182, 273.34516661853604)


0: 384x640 2 persons, 1 tv, 63.5ms
Speed: 0.9ms pre-process, 63.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 persons, 1 tv, 67.4ms


Frame Count -  63
1   [(867.0016694000237, 709.9663028412779), (866.3772085603858, 709.9381826889482), (865.5141919153032, 709.9330652224289), (864.5599118863536, 709.9360818900343), (863.9007813761723, 710.6009897757772), (863.3534177297519, 710.8558562961391), (863.1770763805459, 710.9531335334862), (863.4693208386263, 710.3309429563147), (863.6073893986395, 710.754063313273), (864.0113384106487, 711.5743685925547)]
1  current coordinate (864.0113384106487, 711.5743685925547)
1  previous coordinate (863.6073893986395, 710.754063313273)
2   [(720.4311010460568, 253.87764976877526), (720.0381874259505, 253.83733574683487), (719.2801668131926, 255.72716023236518), (718.7041047058532, 259.01000457146574), (716.5692065191026, 263.4729889892184), (714.1579493613824, 267.7376490264557), (712.6224117574945, 269.9585764693253), (710.7749116933182, 273.34516661853604), (710.4433082901661, 275.8690729765226), (709.7133804985631, 278.7140913413765)]
2  current coordinate (709.7133804985631, 278.

Speed: 1.0ms pre-process, 67.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)

0: 384x640 2 persons, 1 tv, 62.4ms
Speed: 1.0ms pre-process, 62.4ms inference, 1.0ms postprocess per image at shape (1, 3, 640, 640)


1   [(866.3772085603858, 709.9381826889482), (865.5141919153032, 709.9330652224289), (864.5599118863536, 709.9360818900343), (863.9007813761723, 710.6009897757772), (863.3534177297519, 710.8558562961391), (863.1770763805459, 710.9531335334862), (863.4693208386263, 710.3309429563147), (863.6073893986395, 710.754063313273), (864.0113384106487, 711.5743685925547), (864.5122362568666, 710.5687941157914)]
1  current coordinate (864.5122362568666, 710.5687941157914)
1  previous coordinate (864.0113384106487, 711.5743685925547)
2   [(720.0381874259505, 253.83733574683487), (719.2801668131926, 255.72716023236518), (718.7041047058532, 259.01000457146574), (716.5692065191026, 263.4729889892184), (714.1579493613824, 267.7376490264557), (712.6224117574945, 269.9585764693253), (710.7749116933182, 273.34516661853604), (710.4433082901661, 275.8690729765226), (709.7133804985631, 278.7140913413765), (708.1704214281757, 281.0249178885053)]
2  current coordinate (708.1704214281757, 281.0249178885053)
2  

In [4]:
(705.9700773898556, 362.35429410382244)


(705.9700773898556, 362.35429410382244)

In [9]:
if cv2.pointPolygonTest(exitDoorVertices, (705.9700773898556, 362.35429410382244), False) > 0:
    print("Inside")

In [14]:
a = {1: (False, True), 2: True, 3: False}

In [15]:
for bools in a[1]:
    print(bools)

False
True
